In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn import metrics

from src.accelerometer import plot_fourier_transformation, plot_acceleration, plot_feature_columns, accelerometer_feature_engineering
from src.plotting import box_plot_columns
from src.kmeans import kmeans
from src.principal_component_analysis import principal_component_analysis, plot_principal_component_analysis
from src.decision_tree import decision_tree
from src.time_series import median_filter, run_time_series_algorithms
from src.ml_util import evaluate_model

In [ ]:
spiral_accelerations_df = pd.read_csv('spiral_accelerations.csv')
spiral_accelerations_df = spiral_accelerations_df[spiral_accelerations_df['age_group']!=0]

In [ ]:
spiral_drawings_df = pd.read_csv('spiral_drawings.csv')
spiral_drawings_df = spiral_drawings_df[spiral_drawings_df['age_group']!=0]

In [ ]:
spiral_accelerations_df.head(3)

In [ ]:
spiral_accelerations_df.describe()

In [ ]:
spiral_drawings_df.head(3)

In [ ]:
spiral_drawings_df.describe()

In [ ]:
def remove_accelerations_without_drawings(accelerations_df, drawings_df):
    accelerations_copy_df = accelerations_df.copy()
    for uuid in accelerations_df['uuid'].unique():
        max_duration = drawings_df[drawings_df['uuid'] == uuid]['duration'].max()
        drop_df = accelerations_df[(accelerations_df['uuid'] == uuid) & (accelerations_df['duration'] > max_duration)]
        accelerations_copy_df = accelerations_df.drop(drop_df.index)
    return accelerations_copy_df

In [ ]:
spiral_accelerations_df = remove_accelerations_without_drawings(spiral_accelerations_df, spiral_drawings_df)

In [ ]:
first_session_uuid = spiral_accelerations_df['uuid'].unique()[0]
single_session_df = spiral_accelerations_df[spiral_accelerations_df['uuid'] == first_session_uuid]

In [ ]:
plot_acceleration(single_session_df, subplots=False)

In [ ]:
single_session_df = median_filter(single_session_df)
plot_acceleration(single_session_df, subplots=False)

In [ ]:
plot_fourier_transformation(single_session_df, 'test session')

In [ ]:
box_plot_columns(single_session_df)

In [ ]:
plot_acceleration(single_session_df)

In [ ]:
spiral_accelerations_df = median_filter(spiral_accelerations_df)

In [ ]:
spiral_processed_df = accelerometer_feature_engineering(spiral_accelerations_df)

In [ ]:
group_by_keys = ['age_group','subject','hand','uuid']
spiral_data_df = spiral_drawings_df.groupby(group_by_keys)[['first_order_smoothness','second_order_smoothness','thightness','zero_crossing_rate']].agg('first')
spiral_processed_df = spiral_processed_df.merge(spiral_data_df, on=group_by_keys)

In [ ]:
spiral_processed_df.head()

In [ ]:
spiral_processed_df.corr()

SEM is closely related to std -> therefore we drop it

In [ ]:
spiral_processed_df = spiral_processed_df.drop(columns=['x_sem', 'y_sem', 'z_sem', 'mag_sem'])
spiral_processed_df.corr()

In [ ]:
class_key = 'age_group'
plot_feature_columns(spiral_processed_df,'std')
plot_feature_columns(spiral_processed_df,'mean')
plot_feature_columns(spiral_processed_df,'peaks')
plot_feature_columns(spiral_processed_df,'snr')
plot_feature_columns(spiral_processed_df,'sal')
box_plot_columns(spiral_processed_df,class_key, ['first_order_smoothness','second_order_smoothness'])
box_plot_columns(spiral_processed_df,class_key, ['thightness'])
box_plot_columns(spiral_processed_df,class_key, ['zero_crossing_rate'])
box_plot_columns(spiral_processed_df,class_key, ['duration'])

We can see that there is no significant deviation between the two age groups in the following features:
- z_mean
- mag_mean
- z_snr
- mag_snr
- x_sal
- z_sal
- mag_sal

# ML Models

In [ ]:
results = {}

In [ ]:
spiral_processed_df = spiral_processed_df.reset_index(drop=False)
train_df, test_df = train_test_split(spiral_processed_df, test_size=0.10)

In [ ]:
feature_keys = ['x_std','y_std', 'z_std','mag_std','x_mean','y_mean','x_peaks','y_peaks','z_peaks','mag_peaks','x_snr','y_snr','y_sal']

## Apply Kmeans Clustering

In [ ]:
kmeans_model = kmeans(train_df, feature_keys)
kmeans_predicitons = kmeans_model.predict(test_df[feature_keys])
predictions_df = pd.DataFrame()
predictions_df['age_group'] = test_df['age_group']
predictions_df['cluster'] = kmeans_predicitons
predictions_df

In [ ]:
results['kmeans'] = (1-(predictions_df.groupby('age_group')[['cluster']].agg('sem').sum()/2))

## Principal Component Analysis

In [ ]:
principal_components_df = principal_component_analysis(spiral_processed_df, feature_keys)

In [ ]:
plot_principal_component_analysis(spiral_processed_df, principal_components_df, 'age_group', [30,50])

## Decision Tree

In [ ]:
model = decision_tree(train_df, feature_keys)
results['decision_tree'] = evaluate_model(model, test_df[feature_keys],test_df['age_group'])

## Run Time Series Algorithm

In [ ]:
time_series_results = run_time_series_algorithms(spiral_accelerations_df)
results.update(time_series_results)

## ML Results

In [ ]:
results